In [ ]:
def BaseHeuristic(dict_jobs, dict_tasks): 
  list_jobs_sorted = sorted(list(dict_jobs.values()), key=lambda x: x.release_date, reverse=False)

  Schedule = pd.DataFrame(dict(zip(['task', 'start', 'machine', 'operator'], [0, 0, 0, 0])), index=[0])
  Bi = 1

  for idx_job, j in enumerate(list_jobs_sorted):
    if idx_job > 0:
      EndOfLastTask = Schedule.loc[len(Schedule.index) - 1, 'start'] + dict_tasks[Schedule.loc[len(Schedule.index) - 1, 'task']].processing_time
      Bi = np.max([j.release_date, EndOfLastTask])    #Min start time for new job

    for task_nb in j.sequence:
      i = dict_tasks[task_nb]
      Schedule.loc[len(Schedule.index)] = [i.task, Bi, i.machines[0], i.operators[i.machines[0]][0]] 

      Bi = Schedule.loc[len(Schedule.index) - 1, 'start'] + i.processing_time
    
  Schedule = Schedule.drop(0, axis=0, inplace=False)

  return Schedule

In [ ]:
def DueDateHeuristic(dict_jobs, dict_tasks): 
  list_jobs_sorted = sorted(list(dict_jobs.values()), key=lambda x: x.due_date, reverse=False)

  Schedule = pd.DataFrame(dict(zip(['task', 'start', 'machine', 'operator'], [0, 0, 0, 0])), index=[0])
  Bi = 1

  for idx_job, j in enumerate(list_jobs_sorted):
    if idx_job == 0:
      process_time = 0
    else:
      process_time = dict_tasks[Schedule.loc[len(Schedule.index) - 1, 'task']].processing_time

    EndOfLastTask = Schedule.loc[len(Schedule.index) - 1, 'start'] + process_time
    Bi = np.max([j.release_date, EndOfLastTask])    #Min start time for new job

    for task_nb in j.sequence:
      i = dict_tasks[task_nb]
      Schedule.loc[len(Schedule.index)] = [i.task, Bi, i.machines[0], i.operators[i.machines[0]][0]] 

      Bi = Schedule.loc[len(Schedule.index) - 1, 'start'] + i.processing_time
      Bi = np.max([Bi, j.release_date])
    
  Schedule = Schedule.drop(0, axis=0, inplace=False)

  return Schedule

In [ ]:
def PileVersatilityHeuristic(dict_jobs, dict_tasks, dict_machines, dict_operators, dict_costs, alpha, beta): 

  list_jobs_sorted = sorted(list(dict_jobs.values()), key=lambda x: x.due_date, reverse=False)

  #for idx_j, j in enumerate(list_jobs_sorted):
   # for idx_i, i in enumerate(list_jobs_sorted):
    #  if idx_i > idx_j: continue;
     # if i.due_date == j.due_date and j.weight > i.weight:
      #  list_jobs_sorted[idx_i], list_jobs_sorted[idx_j] = list_jobs_sorted[idx_j], list_jobs_sorted[idx_i]


  Schedule = pd.DataFrame(columns=['task', 'start', 'machine', 'operator'])

  for idx_job, j in enumerate(list_jobs_sorted):

    for idx_task, task_nb in enumerate(j.sequence):
      i = dict_tasks[task_nb]
      if idx_task == 0:
        end_date_previous_task_of_same_job = 0
      

      starting_date, first_available_machine, first_available_operator = i.get_schedule(j.release_date, end_date_previous_task_of_same_job, dict_machines, dict_operators, cost_machines = alpha, cost_operators = beta)

      first_available_machine.active_time += list(np.arange(starting_date, starting_date + i.processing_time))
      first_available_operator.active_time += list(np.arange(starting_date, starting_date + i.processing_time))

      Schedule.loc[len(Schedule.index)] = [i.task, starting_date, first_available_machine.machine, first_available_operator.operator] 


      end_date_previous_task_of_same_job = starting_date + i.processing_time + 1
    
  score = (compute_cost(dict_jobs, dict_tasks, dict_costs, Schedule))

  return Schedule, score